In [68]:
from rdfpandas.graph import to_graph
import pandas as pd
import rdflib
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from shexer.shaper import Shaper
from shexer.consts import NT, TURTLE

In [69]:
fdp_index = "https://index.vp.ejprarediseases.org/?format=ttl"

fdpGraph = rdflib.Graph()
fdpGraph.parse(fdp_index, format="ttl")

begin


In [70]:
print(fdpGraph.serialize(format="ttl"))

,uuid,clientUrl,state,registrationTime,modificationTime
0,cc94cde5-74c5-4256-bbf1-50ec3fad7319,https://w3id.org/duchenne-fdp,ACTIVE,2023-02-25T14:41:17.281Z,2023-05-12T10:50:06.647Z
1,9e47792b-1540-4f4f-9748-db030f0787b1,http://w3id.org/ern-euro-nmd-fdp,ACTIVE,2023-02-27T08:15:55.984Z,2023-05-10T00:22:18.394Z
2,27d40e18-e1c6-45b1-940d-59105901bae8,https://w3id.org/ctsr-fdp,ACTIVE,2023-02-27T08:23:29.161Z,2023-05-10T00:22:18.237Z
3,255d928d-9f0b-4987-af9e-be9bf7d13392,https://w3id.org/smartcare-fdp,ACTIVE,2023-02-27T09:34:25.473Z,2023-05-10T00:22:18.236Z
4,d06f3341-bd37-426d-ad69-59fbf66ad043,https://w3id.org/fairvasc-fdp/,ACTIVE,2023-03-15T15:14:48.763Z,2023-05-11T13:06:59.397Z


# Issue 1
[ttl rendering of https://index.vp.ejprarediseases.org/](https://index.vp.ejprarediseases.org/?format=ttl) is not identical to html output on https://index.vp.ejprarediseases.org/

Those URL's need to be scraped which I will do below


In [72]:
url = 'https://index.vp.ejprarediseases.org'

headers = {'Accept': 'text/html'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content.decode('UTF-8'), 'html.parser')
text = soup.get_text()
for link in soup.find_all('a'):
    print(link.get('href'))

text

https://w3id.org/duchenne-fdp/catalog/ce01b2e8-3a1d-434e-b5e2-98e3a8d78a5b
https://w3id.org/duchenne-fdp/dataset/3a2b5a69-b36f-4db9-983d-a1e410ba9c71
http://w3id.org/ern-euro-nmd-fdp/catalog/9e38450d-89cb-4f78-ac50-0f048a633443
http://w3id.org/ern-euro-nmd-fdp/dataset/5d6f2a12-4d8d-4f3a-ad83-b6e8f0fe9dc5
https://w3id.org/ctsr-fdp/catalog/87acc8da-4885-4b73-85f4-7caaa062600b
https://w3id.org/ctsr-fdp/dataset/b5bc04b6-bd1a-49f3-a5b3-a329b3ce6b9c
https://w3id.org/ctsr-fdp/catalog2/8fa78605-8025-4e0c-bf63-4da64ee727a2
https://w3id.org/ctsr-fdp/catalog2/c1934541-a329-4139-96a2-a784c602b9da
https://w3id.org/ctsr-fdp/site/d42ffcf8-5d87-47e2-a1dd-8bd00294c75c
https://w3id.org/ctsr-fdp/site/19b1021e-7b67-454c-9a61-0d511854000f
https://w3id.org/ctsr-fdp/site/070a6704-67c5-4318-8643-5741e7222343
https://w3id.org/ctsr-fdp/contact_individual/9f1a623c-a114-40e3-866e-ab61bcb1aa57
https://w3id.org/ctsr-fdp/contact_individual/9f1a623c-a114-40e3-866e-ab61bcb1aa57
https://w3id.org/ctsr-fdp/contact_organi

## Issue 2
The human-readable rendering of the FDP index can't be access programmatically, because it needs Javascript

In [74]:
# Launch a headless browser
driver = webdriver.Chrome()
driver.get(url)

# Wait for the page to fully load
driver.implicitly_wait(10)  # wait for up to 10 seconds

# Get the HTML content of the page
html = driver.page_source

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Close the browser


# Use BeautifulSoup to extract information from the page
htmls = []
for link in soup.find_all('a'):
    if "entry" in link.get('href'):
        print(link.get('href'))
        driver = webdriver.Chrome()
        driver.get(url+link.get('href'))
        driver.implicitly_wait(10)
        subhtml = driver.page_source
        sub_soup = BeautifulSoup(subhtml, 'html.parser')
        for sublink in sub_soup.find_all('a'):
            if "https" in sublink.get('href'):
                print(sublink.get('href'))
                fdpGraph.parse(sublink.get('href'), format="ttl")
                break

driver.quit()




Fetching linked data from the web
Fetching: http://bioregistry.io/sio:SIO_001001
Could not parse: http://bioregistry.io/sio:SIO_001001
Fetching: http://purl.obolibrary.org/obo/HP_0002019
Could not parse: http://purl.obolibrary.org/obo/HP_0002019
Fetching: http://purl.obolibrary.org/obo/NCIT_C121238
Fetching: https://w3id.org/duchenne-fdp/catalog/ce01b2e8-3a1d-434e-b5e2-98e3a8d78a5b#identifier
Fetching: http://purl.org/dc/terms/hasPart
Fetching: https://w3id.org/duchenne-fdp/dataservice/017be60d-8e5e-41c9-b4e6-3dd572285cfa/metrics/445c0a70d1e214e545b261559e2842f4
Could not parse: https://w3id.org/duchenne-fdp/dataservice/017be60d-8e5e-41c9-b4e6-3dd572285cfa/metrics/445c0a70d1e214e545b261559e2842f4
Fetching: https://en.wikipedia.org/wiki/All_rights_reserved
Could not parse: https://en.wikipedia.org/wiki/All_rights_reserved
Fetching: http://w3id.org/ern-euro-nmd-fdp/profile/77aaad6a-0136-4c6e-88b9-07ffccd0ee4c
Fetching: http://purl.obolibrary.org/obo/NCIT_C94198
Fetching: https://w3id.org

In [75]:
uris = {"fetched": [], "not_fetched": []}

print("Fetching linked data from the web")
for node in fdpGraph.all_nodes():
    if isinstance(node, rdflib.URIRef):
        try:

            print("Fetching: " + str(node))
            fdpGraph.parse(node)
            uris["fetched"].append(node)
        except:
            uris["not_fetched"].append(node)
            print("Could not parse: " + str(node))
            continue

<Graph identifier=N10ca4d325cfa419597c09f6f16022f75 (<class 'rdflib.graph.Graph'>)>

In [76]:
fdpGraph.serialize(format="ttl", destination="fdp_index.ttl")

fetched:188, not fetched:183


In [77]:
print("fetched:"+ str(len(uris["fetched"]))+", not fetched:"+ str(len(uris["not_fetched"])))

In [78]:
shex_target_file = "fdp_index.shex"
target_classes = [
    "http://www.w3.org/ns/dcat#:Resource",
    "http://www.w3.org/ns/dcat#DataService",
    "https://w3id.org/fdp/fdp-o#MetadataService",
    "https://w3id.org/fdp/fdp-o#FAIRDataPoint"
]

shaper = Shaper(target_classes=target_classes,
                rdflib_graph=fdpGraph,
                input_format=TURTLE,
                )  # Default rdf:type

shaper.shex_graph(output_file="fdpSchema.shex")

'PREFIX : <http://weso.es/shapes/>\nPREFIX brick: <https://brickschema.org/schema/Brick#>\nPREFIX csvw: <http://www.w3.org/ns/csvw#>\nPREFIX dc: <http://purl.org/dc/elements/1.1/>\nPREFIX dcat: <http://www.w3.org/ns/dcat#>\nPREFIX dcam: <http://purl.org/dc/dcam/>\nPREFIX doap: <http://usefulinc.com/ns/doap#>\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nPREFIX geo: <http://www.opengis.net/ont/geosparql#>\nPREFIX odrl: <http://www.w3.org/ns/odrl/2/>\nPREFIX org: <http://www.w3.org/ns/org#>\nPREFIX prof: <http://www.w3.org/ns/dx/prof/>\nPREFIX prov: <http://www.w3.org/ns/prov#>\nPREFIX qb: <http://purl.org/linked-data/cube#>\nPREFIX schema: <https://schema.org/>\nPREFIX sh: <http://www.w3.org/ns/shacl#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX sosa: <http://www.w3.org/ns/sosa/>\nPREFIX ssn: <http://www.w3.org/ns/ssn/>\nPREFIX time: <http://www.w3.org/2006/time#>\nPREFIX vann: <http://purl.org/vocab/vann/>\nPREFIX void: <http://rdfs.org/ns/void#>\nPREFIX wgs: <https://ww

In [79]:
!cat fdpSchema.shex

PREFIX : <http://weso.es/shapes/>

PREFIX brick: <https://brickschema.org/schema/Brick#>

PREFIX csvw: <http://www.w3.org/ns/csvw#>

PREFIX dc: <http://purl.org/dc/elements/1.1/>

PREFIX dcat: <http://www.w3.org/ns/dcat#>

PREFIX dcam: <http://purl.org/dc/dcam/>

PREFIX doap: <http://usefulinc.com/ns/doap#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>

PREFIX geo: <http://www.opengis.net/ont/geosparql#>

PREFIX odrl: <http://www.w3.org/ns/odrl/2/>

PREFIX org: <http://www.w3.org/ns/org#>

PREFIX prof: <http://www.w3.org/ns/dx/prof/>

PREFIX prov: <http://www.w3.org/ns/prov#>

PREFIX qb: <http://purl.org/linked-data/cube#>

PREFIX schema: <https://schema.org/>

PREFIX sh: <http://www.w3.org/ns/shacl#>

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX sosa: <http://www.w3.org/ns/sosa/>

PREFIX ssn: <http://www.w3.org/ns/ssn/>

PREFIX time: <http://www.w3.org/2006/time#>

PREFIX vann: <http://purl.org/vocab/vann/>

PREFIX void: <http://rdfs.org/ns/void#>

PREFIX wgs: <https://www

{'DataService': '{\n   dcat:endpointURL  IRI  ;                                    # 100.0 %\n            # 62.5 % obj: @:MetadataService. Cardinality: {1}\n            # 62.5 % obj: @:FAIRDataPoint. Cardinality: {1}\n            # 62.5 % obj: @:DataService. Cardinality: {1}\n   rdf:type  [dcat:DataService]  ;                             # 100.0 %\n   rdf:type  [dcat:Resource]  ;                                # 100.0 %\n   dcterms:language  IRI  ;                                    # 100.0 %\n   rdfs:label  xsd:string  ;                                   # 100.0 %\n   dcterms:accessRights  IRI  ;                                # 100.0 %\n   ns6:SIO_000628  IRI  {2};                                   # 100.0 %\n   dcterms:conformsTo  IRI  ;                                  # 100.0 %\n   ns1:metadataModified  xsd:dateTime  ;                       # 100.0 %\n   ns1:metadataIdentifier  IRI  ;                              # 100.0 %\n   dcterms:description  xsd:string  ;                    

In [80]:
file = open("fdpSchema.shex", "r")
fdp_shex = file.read()
fdp_shex

{
   dcat:endpointURL  IRI  ;                                    # 100.0 %
            # 62.5 % obj: @:MetadataService. Cardinality: {1}
            # 62.5 % obj: @:FAIRDataPoint. Cardinality: {1}
            # 62.5 % obj: @:DataService. Cardinality: {1}
   rdf:type  [dcat:DataService]  ;                             # 100.0 %
   rdf:type  [dcat:Resource]  ;                                # 100.0 %
   dcterms:language  IRI  ;                                    # 100.0 %
   rdfs:label  xsd:string  ;                                   # 100.0 %
   dcterms:accessRights  IRI  ;                                # 100.0 %
   ns6:SIO_000628  IRI  {2};                                   # 100.0 %
   dcterms:conformsTo  IRI  ;                                  # 100.0 %
   ns1:metadataModified  xsd:dateTime  ;                       # 100.0 %
   ns1:metadataIdentifier  IRI  ;                              # 100.0 %
   dcterms:description  xsd:string  ;                          # 100.0 %
   dcterms:titl

In [81]:
shapes = dict()
shapeName = None
with open('fdpSchema.shex', 'r') as f:
    for line in f:
        print(line)
        if line.startswith(":"):
            shapeName = line.replace(":", "").replace("\n", "")
            continue
        if shapeName is not None:
            shapes[shapeName] = shapes.get(shapeName, "") + line
        # code to do something with the list of numbers goes here
shapes

,Property Name,Value Type,Value Constraints
0,dcat:endpointURL,IRI,; # 100.0
1,#,62.5,% obj: @:MetadataService. Cardinality:
2,#,62.5,% obj: @:FAIRDataPoint. Cardinality:
3,#,62.5,% obj: @:DataService. Cardinality:
4,rdf:type,[dcat:DataService],; # 100.0
5,rdf:type,[dcat:Resource],; # 100.0
6,dcterms:language,IRI,; # 100.0
7,rdfs:label,xsd:string,; # 100.0
8,dcterms:accessRights,IRI,; # 100.0
9,ns6:SIO_000628,IRI,{2}; # 100.0


In [81]:
print(shapes['DataService'])

In [ ]:
import pandas as pd

# Split the input string into lines
lines = shapes['DataService'].split('\n')

# Extract the property name, value type, value constraints, and percentage from each line
data = []
for line in lines:
    line = line.strip()
    if not line or line.startswith('{') or line.startswith('}'):
        continue
    parts = line.split()
    prop_name = parts[0]
    value_type = parts[1]
    value_constraints = ' '.join(parts[2:-1]) if len(parts) > 2 else ''
    percentage = parts[-1]
    data.append((prop_name, value_type, value_constraints))

# Convert the list of tuples to a pandas dataframe
df = pd.DataFrame(data, columns=['Property Name', 'Value Type', 'Value Constraints'])

# Print the dataframe
df

In [ ]:
for subject in fdpGraph.subjects():
    print(subject)